# Imports

In [43]:
import deeplake
import cv2
import os

In [10]:
ds_downloaded = deeplake.load("hub://activeloop/kuzushiji-kanji")

|

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/kuzushiji-kanji



\

hub://activeloop/kuzushiji-kanji loaded successfully.



In [11]:
type(ds_downloaded)

deeplake.core.dataset.deeplake_cloud_dataset.DeepLakeCloudDataset

In [12]:
ds_downloaded.visualize()

In [13]:
ds_downloaded[0]

Dataset(path='hub://activeloop/kuzushiji-kanji', read_only=True, index=Index([0]), tensors=['images', 'labels'])

In [42]:
img0 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/0.png", cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/1.png", cv2.IMREAD_GRAYSCALE)
img_kkanji = cv2.imread("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2/U+4C61/e6eeb23552e1a21c.png", cv2.IMREAD_GRAYSCALE)
img0 = cv2.resize(img0, (64, 64))
img1 = cv2.resize(img1, (64, 64))
img0 = img0 / 255
img1 = img1 / 255

# Morphology opening and closing
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
img_kkanji_opened = cv2.morphologyEx(img_kkanji, cv2.MORPH_OPEN, kernel)
img_kkanji_closed = cv2.morphologyEx(img_kkanji_opened, cv2.MORPH_CLOSE, kernel)

# Save the thresholded image
cv2.imwrite("/home/beav3r/Bachelor_work/Bachelor_work/data/追加/準備ができていない/Unicode/thresholdedTest.png", img_kkanji_closed)

img_kkanji_closed.shape

(64, 64)

In [46]:
# Count amount of files in subsets and write into txt file

path = "/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji/kkanji2"

def count_files(path):
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            #Save to txt file
            with open("/home/beav3r/Bachelor_work/Bachelor_work/data/kkanji_classes_distribution.txt", "a") as file:
                file.write(dir + " " + str(len(os.listdir(os.path.join(root, dir)))) + "\n")
        
count_files(path)



In [ ]:
# Sort the classes by amount of files
with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution.txt", "r") as file:
    lines = file.readlines()
    lines.sort(key=lambda x: int(x.split()[1]), reverse=True)
    with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
        for line in lines:
            file.write(line)

In [47]:
# Remote duplicates from the sorted file
with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "r") as file:
    lines = file.readlines()
    lines = list(dict.fromkeys(lines))
    with open("/home/beav3r/Bachelor_work/Bachelor_work/kkanji_classes_distribution_sorted.txt", "w") as file:
        for line in lines:
            file.write(line)